In [5]:
import pandas as pd
import tensorflow as tf


In [17]:
df = pd.read_csv('train.csv')

In [18]:
df.shape

(20800, 5)

In [21]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [22]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
df = df.dropna()

In [24]:
df.shape

(18285, 5)

In [26]:
x = df.drop('label' , axis = 1)
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [28]:
y = df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [29]:
print(x.shape , y.shape)

(18285, 4) (18285,)


In [31]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [32]:
Vocab = 5000

In [35]:
messages = x.copy()

In [36]:
messages['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [37]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [38]:
messages


,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [56]:
messages.reset_index(inplace=True)

In [39]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
del(review)

In [58]:
messages.shape


(18285, 5)

In [59]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [60]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [62]:
onehot_repr=[one_hot(words,Vocab)for words in corpus]
onehot_repr

[[2831, 510, 24, 2915, 3272, 2121, 3907, 4015, 2622, 4700],
 [1474, 4863, 2523, 1309, 2546, 140, 1433],
 [3206, 2040, 1212, 4490],
 [1983, 384, 4282, 2452, 3733, 1528],
 [2617, 2546, 4209, 2860, 257, 708, 2546, 2625, 4425, 1168],
 [4339,
  3833,
  1693,
  4865,
  2189,
  1791,
  208,
  2495,
  686,
  2210,
  4346,
  4439,
  1945,
  2701,
  1433],
 [3125, 2386, 2491, 2062, 186, 1599, 1217, 2060, 3646, 1046, 4523],
 [1087, 1294, 2615, 1953, 3424, 199, 1791, 2615, 3646, 1046, 4523],
 [2507, 4139, 2210, 2905, 4741, 1221, 2099, 1436, 1791, 4738],
 [1582, 3587, 3479, 3218, 2687, 3507, 2798, 1609],
 [4058, 1279, 2433, 559, 1066, 987, 3890, 2609, 2343, 268, 1403],
 [2452, 3845, 3272, 1221, 1791, 3424],
 [2797, 4255, 2731, 118, 2409, 4479, 114, 4860, 4089],
 [1441, 1650, 2819, 1429, 2708, 3277, 3400, 3646, 1046, 4523],
 [2555, 1072, 3113, 4829, 3267, 3646, 1046, 4523],
 [3695, 1449, 4331, 2832, 4743, 2902, 3636, 3149, 2646, 3460],
 [4827, 1803, 4863],
 [1187, 1312, 3242, 2768, 1791, 1514, 4957,

# Embedding

In [63]:
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2831  510   24 ...    0    0    0]
 [1474 4863 2523 ...    0    0    0]
 [3206 2040 1212 ...    0    0    0]
 ...
 [1919  525  470 ...    0    0    0]
 [2027 3424  711 ...    0    0    0]
 [4265 3560 1865 ...    0    0    0]]


In [64]:
## Creating model
embedding_vector_features= 40 ##features representation
model=Sequential()
model.add(Embedding(Vocab,embedding_vector_features,input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 150)               114600    
                                                                 
 dense (Dense)               (None, 1)                 151       
                                                                 
Total params: 314751 (1.20 MB)
Trainable params: 314751 (1.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [66]:
len(embedded_docs),y.shape

(18285, (18285,))

In [67]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [68]:
X_final.shape,y_final.shape

((18285, 15), (18285,))

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=19)

In [70]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 21s 66ms/step - loss: 0.2969 - accuracy: 0.8573 - val_loss: 0.1995 - val_accuracy: 0.9152
Epoch 2/10
200/200 [==============================] - 3s 13ms/step - loss: 0.1447 - accuracy: 0.9443 - val_loss: 0.1937 - val_accuracy: 0.9160
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 0.1050 - accuracy: 0.9607 - val_loss: 0.2372 - val_accuracy: 0.9089
Epoch 4/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0758 - accuracy: 0.9727 - val_loss: 0.2528 - val_accuracy: 0.9125
Epoch 5/10
200/200 [==============================] - 2s 12ms/step - loss: 0.0535 - accuracy: 0.9812 - val_loss: 0.3509 - val_accuracy: 0.9078
Epoch 6/10
200/200 [==============================] - 2s 11ms/step - loss: 0.0398 - accuracy: 0.9881 - val_loss: 0.3754 - val_accuracy: 0.9112
Epoch 7/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0319 - accuracy: 0.9906 - val_loss: 0.3137 - val_accuracy: 0.9120


In [72]:
y_pred= model.predict(X_test)

172/172 [==============================] - 0s 2ms/step


In [73]:
y_pred= np.where(y_pred > 0.6 , 1 , 0)

In [74]:
from sklearn.metrics import confusion_matrix

In [75]:
confusion_matrix(y_test,y_pred)

array([[2902,  230],
       [ 336, 2018]])

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8968282901932191

In [77]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      3132
           1       0.90      0.86      0.88      2354

    accuracy                           0.90      5486
   macro avg       0.90      0.89      0.89      5486
weighted avg       0.90      0.90      0.90      5486

